In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
data=pd.read_csv("../input/skylarfreedman/Documents-Table 1.csv" ,sep=';' , engine='python')

In [3]:
data.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-004567394_00861,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00861.jpg,1,[D104 ENCL]\r\nNorth Carolina\r\nThis indentur...,/11400/transcriptions/NMAAHC-004567394_00861.txt,\N,\N,11400,Contracts,NaN,NaN
1,NMAAHC-004567394_00957,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00957.jpg,1,[E 11 ENCL] \r\n\r\n\r\nState of North Carolin...,/11400/transcriptions/NMAAHC-004567394_00957.txt,\N,\N,11400,Contracts,NaN,NaN
2,NMAAHC-004567394_00958,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00958.jpg,1,"near Lizzie Etheridge's gate, thence South 56 ...",/11400/transcriptions/NMAAHC-004567394_00958.txt,\N,\N,11400,Contracts,NaN,NaN
3,NMAAHC-004567395_00439,Complete,https://transcription.si.edu/transcribe/11406/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11406/image/NMAAHC-004567395_00439.jpg,1,Copy\r\n\r\nAgreement of Apprenticeship\r\nBu ...,/11406/transcriptions/NMAAHC-004567395_00439.txt,\N,\N,11406,Contracts,NaN,NaN
4,NMAAHC-004567395_00440,Complete,https://transcription.si.edu/transcribe/11406/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11406/image/NMAAHC-004567395_00440.jpg,1,[H 132 ENCL] \r\n\r\nCopy\r\n\r\nAgreement of ...,/11406/transcriptions/NMAAHC-004567395_00440.txt,\N,\N,11406,Contracts,NaN,NaN


In [4]:
data.shape

(278926, 15)

In [5]:
data["Category"].unique()

array(['Contracts', 'Duplicate Scan', 'Index', 'Letters/Reports',
       'Letters/Reports ', 'Miscellaneous', 'Oaths/Pardons',
       'Orders/Circulars', 'Records - Court', 'Records - Courts',
       'Records - Education', 'Records - Financial', 'Records - Military',
       'Records - Personnel', 'Records - Property', 'Records - Rations',
       'Records - Transportation', 'Records - Vital', 'Records -Court',
       'Reports', 'XX - Blank page', 'XX - Blank Page',
       'XX - Duplicate Scab', 'XX - Duplicate Scan', 'XX - Metadata',
       'XX - Title page', 'XX - Title Page', 'XX- Duplicate Scan'],
      dtype=object)

In [6]:
contact=data[data["Category"]=="Contracts"]
order=data[data["Category"]=="Orders/Circulars"]
index=data[data["Category"]=="Index"]
recordE=data[data["Category"]=="Records - Education"]
recordF=data[data["Category"]=="Records - Financial"]
recordC=data[data["Category"]=="Records - Court"]

In [7]:
contact.head(2)

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-004567394_00861,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00861.jpg,1,[D104 ENCL]\r\nNorth Carolina\r\nThis indentur...,/11400/transcriptions/NMAAHC-004567394_00861.txt,\N,\N,11400,Contracts,NaN,NaN
1,NMAAHC-004567394_00957,Complete,https://transcription.si.edu/transcribe/11400/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/11400/image/NMAAHC-004567394_00957.jpg,1,[E 11 ENCL] \r\n\r\n\r\nState of North Carolin...,/11400/transcriptions/NMAAHC-004567394_00957.txt,\N,\N,11400,Contracts,NaN,NaN


In [8]:
contact.shape , index.shape,recordF.shape, recordE.shape, recordC.shape, order.shape

((23731, 15), (5422, 15), (3599, 15), (13487, 15), (6117, 15), (7668, 15))

In [9]:
contact.to_csv("contact.csv",index=False)
index.to_csv("index.csv",index=False)
recordF.to_csv("recordF.csv",index=False)
recordE.to_csv("recordE.csv",index=False)
recordC.to_csv("recordC.csv",index=False)
order.to_csv("order.csv",index=False)